# 使用 Tool 

1. 安装对应 Tool
2. 确定要执行的 Tool
3. 执行 Tool
4. 整合后的结果

In [5]:
expression_evaluator_tool = {
        'type': 'function',
        'function': {
          'name': 'expression_evaluator',
          'description': 'Evaluates a mathematical expression following the PEMDAS/BODMAS order of operations.',
          'parameters': {
            'type': 'object',
            'properties': {
              'expression': {
                'type': 'string',
                'description': 'The mathematical expression to evaluate. The expression can include integers, decimals, parentheses, and the operators +, -, *, and /.',
              }
            },
            'required': ['expression'],
          },
        },
      }

import numexpr as ne

def expression_evaluator(expression = ""):
    try:
        result = ne.evaluate(expression)
        return f"Answer to {expression} is {result}"
    except Exception as e:
        return str(e)

In [12]:
import ollama
tool_history = []

OLLAMA_HOST="http://localhost:6060"

def tool_calling(query):
    
    messages = [{'role': 'user', 'content': query}]

    # response = ollama.chat(
    response = ollama.chat(    
        model='llama3.1:latest',
        messages=messages,
        tools=[expression_evaluator_tool,],
    )

    print(response['message'])
    
    messages.append(response['message'])

    if not response['message'].get('tool_calls'):
        print("The model didn't use the function. Its response was:")
        print(response['message']['content'])

    if response['message'].get('tool_calls'):
        available_functions = {
            'expression_evaluator' : expression_evaluator
            # 'wikipedia' : wikipedia,
            # 'weather_forecast' : weather_forecast
        }
    for tool in response['message']['tool_calls']:

        function_to_call = available_functions[tool['function']['name']]
        args = tool['function']['arguments'].values()
        function_response = function_to_call(*args)

        print("\nTool Response:\n", function_response)

        messages.append({'role': 'tool', 'content': function_response})

    final_response = ollama.chat(model='llama3.1', messages=messages)

    return final_response['message']['content']

In [14]:
query = 'What is (100+2)**3'
result = tool_calling(query)
print("\n\nAnswer:\n", result)

{'role': 'assistant', 'content': '(100 + 2) ** 3 = 102 ** 3\n= 1064848'}
The model didn't use the function. Its response was:
(100 + 2) ** 3 = 102 ** 3
= 1064848


KeyError: 'tool_calls'